In [ ]:
import sys
import pandas as pd
sys.path.append('../')
from pepq.io import load_json
from pepq.data import build_data
core = load_json('../data/testcase/core56.json')
dockq_core = pd.read_csv('../data/testcase/dockq_core56.csv')

train = load_json('../data/train/train.json')
dockq_train = pd.read_csv('../data/train/dockq_train.csv')

core = build_data(core, dockq_core)
train = build_data(train, dockq_train)



In [ ]:
from pepq.pepq import PepQ

from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(train, test_size=0.2, random_state=42)
X_train, X_val = train_set.drop(columns=['label', "dockq"]), val_set.drop(columns=['label', "dockq"])
y_train, y_val = train_set['dockq'], val_set['dockq']


cfg = PepQ.make_default_preprocessor_cfg(
            variance_threshold=0.00,
            scaler="minmax",
            return_dataframe=True,
        )

pepq = PepQ(preprocessor_cfg=cfg, target_key="dockq", cache_calibration=False)
pepq.fit((X_train, y_train), (X_val, y_val))

pepq.calibrate_threshold(
    X_eval=X_val,
    y_eval=y_val,
    scan_kwargs={"score_type": "fbeta", "beta": 4, "use_abs_pearson": False},
    pick_kwargs={
        "metric_col": "pearson_r_in_domain",
        "mode": "target",
        "target_coverage": 0.70,
    },
    store_tradeoff_df=False,   
)

pepq.save("../pepq/pepq", compress=("xz", 9), compact=True)


In [ ]:
from pepq.pepq import PepQ
from pepq.io import load_json
from pepq.data import build_data

pipe = PepQ.load('../pepq/pepq')
off = load_json('../data/testcase/off-targets.json')
off = build_data(off, None)

conf = pipe.predict_confident(off, threshold=None)
y_pred = conf.confident_predictions
y_pred

In [ ]:
from pepq.pepq import PepQ
from pepq.io import load_json
from pepq.data import build_data

pipe = PepQ.load() # default load built-in model
off = load_json('../data/testcase/off-targets.json')
off = build_data(off, None)

conf = pipe.predict_confident(off, threshold=None)
y_pred = conf.confident_predictions
y_pred

In [ ]:
from pepq.metrics.plot_regression import plot_regression_report

fig, _ =plot_regression_report(y_true=y_val, y_pred=pipe.predict(X_val), include=["pred_vs_true", "resid_hist", "resid_vs_pred", "qq", "density", "tolerance"])
fig